In [2]:
from IPython.display import display
import re
from cameo import models
from cameo.strain_design import pathway_prediction
from os.path import dirname, join, pardir
from cobra.io import read_sbml_model
from dfba import DfbaModel, ExchangeFlux, KineticVariable

ModuleNotFoundError: No module named 'dfba'

In [2]:
docker



NameError: name 'docker' is not defined

In [1]:
docker run --rm -it davidtourigny/dfba:latest

SyntaxError: invalid syntax (1901972628.py, line 1)

In [8]:
pip install dfba

  Using cached dfba-0.1.8.tar.gz (63 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached symengine-0.10.0-cp311-cp311-win_amd64.whl (15.6 MB)
Failed to build dfba
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for dfba (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [40 lines of output]
      C:\Users\pauli\AppData\Local\Temp\pip-build-env-ikr1nwzv\overlay\Lib\site-packages\setuptools\config\setupcfg.py:293: _DeprecatedConfig: Deprecated config in `setup.cfg`
      !!
      
              ********************************************************************************
              The license_file parameter is deprecated, use license_files instead.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ********************************************************************************
      
      !!
        parsed = self.parsers.get(option_name, lambda x: x)(value)
      running bdist_wheel
     

In [ ]:
model = read_sbml_model("yeast8.xml")

In [ ]:
biomass_objective = model.optimize().objective_value
model.reactions.GROWTH.lower_bound = biomass_objective*0.5 #set the minimum biomass to be 50% of the flux
print("Biomass bounds now are:", model.reactions.GROWTH.bounds)

In [ ]:
model.objective = model.reactions.r_1180
model.optimize()